# Step One - Getting Data on Disk

This demo will begin by using the familiar and accessible weatherbench dataset (see https://github.com/pangeo-data/WeatherBench), and follow the examples provided at https://github.com/nci/NCI_WeatherBench but using EDIT along the way. 

Examples will use low-resolution data, but the data is still really big. It's going to be easier to do this on midrange or HPC than typical workstation setups, although a high-end workstation will be sufficient for getting the general idea.

Readers are strongly recommented to start by working through the non-EDIT version of the tutorials in order to have a strong grasp of what is going on with the ML **before** tackling the concepts in EDIT. The final EDIT examples are well-structured and simpler, but it's going to be difficult to learn what's going on in EDITland without seeing what a more bare-bones approach looks like.

Weatherbench2 provides some updates, and moved to a cloud-native architecture. This may suit those working in a cloud environment, but is more complex for those working on a workstation or HPC environment where it is easier to have the files replicated to disk first. 



## Data Download and layout on disk

If you don't already have a replication of the dataset, the zipped data file is 270GB and can be downloaded thusly:

> wget "https://dataserv.ub.tum.de/s/m1524895/download?path=%2F5.625deg&files=all_5.625deg.zip" -O all_5.625deg.zip

The process of unzipping the downloaded file is left as an exercise to the reader. The unzipped data is 471GB. Once downloaded and unpacked, proceed.

If you are working at NCI, the data is already available on disk as a data collection.


In [8]:
import os
wbench_data_dir = '/g/data/wb00/NCI-Weatherbench/5.625deg'
!ls $wbench_data_dir

10m_u_component_of_wind  potential_vorticity	       total_cloud_cover
10m_v_component_of_wind  relative_humidity	       total_precipitation
2m_temperature		 specific_humidity	       u_component_of_wind
constants		 temperature		       v_component_of_wind
geopotential		 toa_incident_solar_radiation  vorticity


# Loading Data the Old Fashioned Way

https://github.com/pangeo-data/WeatherBench/blob/master/notebooks/3-cnn-example.ipynb shows some examples of loading data the old-fashioned way. In short, you need to write the looping code to walk the filesystem, open things file-by-file, and manually do any reprocessing.

# Loading Data with EDIT

See https://bureau-sig.nci.org.au/edit/EDIT_101/Data/ for more documentation.

EDIT provides a standardised interface to data. Code is already written to nicely interface to the full ERA5 that's on disk at NCI (not just the 5.625 degree data from weatherbench). We'll start by taking a look at the EDIT ERA5 interface, then walk through the process of connecting to the low-res dataset to see how to connect to a new data source. 

In [2]:
import edit_archive_NCI
import edit.data
import edit.pipeline
import edit
import edit_archive_NCI
edit.data.archive.NCI?

WOOOOOOHOOOOO


/scratch/kd24/tjl548/testeditableedit/data/src/edit/data/archive/root.py:24: UserWarning: `ROOT_DIRECTORIES` not found underneath `edit.data.archive`, either archives are not installed or misconfigured. Root Directories cannot be changed. 
  warnings.warn(
/scratch/kd24/tjl548/testeditableedit/data/src/edit/data/__init__.py:142: FutureWarning: `edit` is under heavy development and may not continue to be supported.
  __python_warnings.warn(
/scratch/kd24/tjl548/testeditableedit/nci/src/edit_archive_NCI/ERA5.py:37: AccessorRegistrationWarning: Registration of archive <class 'edit_archive_NCI.ERA5.ERA5'> under name 'ERA5' is overriding a preexisting archive with the same name.
  @register_archive("ERA5", sample_kwargs=dict(variable="2t"))


Type:        module
String form: <module 'edit_archive_NCI' from '/scratch/kd24/tjl548/testeditableedit/nci/src/edit_archive_NCI/__init__.py'>
File:        /scratch/kd24/tjl548/testeditableedit/nci/src/edit_archive_NCI/__init__.py
Docstring:  
National Computing Infrastructure specific Indexes

| Name        | Description |
| :---        |       ----: |
| [ERA5][edit_archive_NCI.ERA5]                | ECWMF ReAnalysis v5       |
| [ACCESS][edit_archive_NCI.ACCESS]            | Australian Community Climate and Earth-System Simulator       |
| [AGCD][edit_archive_NCI.AGCD]                | Australian Gridded Climate Data        |
| [BRAN][edit_archive_NCI.BRAN]                | Bluelink ReANalysis        |
| [OceanMaps][edit_archive_NCI.OceanMaps]      | Ocean Modelling and Analysis Prediction System        |
| [MODIS][edit_archive_NCI.MODIS]              | MODerate resolution Imaging Spectroradiometer       |
| [Himiwari][edit_archive_NCI.Himiwari]        | Himiwari 8/9 satellite data  

In [4]:
!pwd

/home/548/tjl548


In [3]:
edit.data.archive?

Type:        module
String form: <module 'edit.data.archive' from '/scratch/kd24/tjl548/testeditableedit/data/src/edit/data/archive/__init__.py'>
File:        /scratch/kd24/tjl548/testeditableedit/data/src/edit/data/archive/__init__.py
Docstring:  
Provide [Index][edit.data.ArchiveIndex] for known and widely used archived data sources.

These [Indexes][edit.data.ArchiveIndex] allow a user to retrieve data with only a date after being initialised.

More archives can be added by wrapping a class with [register_archive][edit.data.archive.register_archive]

!!! Warning
    `edit.data` contains no archives itself, and require additional modules to define them.

    Currently the following exist,
    ```
     - NCI
     - UKMO
    ```

!!! Note
    If setup correctly, any registered archive will be automatically imported if detected to be on the appropriate system.
    So, there may be no need to explicity import it.

In [16]:
var=['u', 'v']  # Note - there is no really straightforward way to just list the variables in the archive
                # However, mismatches will cause edit to list what's available with a "did you mean" prompt
                # A specific listing function should be added in future.
UandV = edit.data.archive.ERA5(var)
UandV

ERA5
	Description                    ECWMF ReAnalysis v5
		 range                          '1970-current'
		 Documentation                  'https://confluence.ecmwf.int/display/CKB/ERA5%3A+data+documentation'


	Initialisation                 
		 level_value                    None
		 product                        'reanalysis'
		 variables                      ['u', 'v']
	Transforms                     
		 StandardCoordinateNames        {'latitude': "['lat', 'Latitude', 'yt_ocean', 'yt']", 'longitude': "['lon', 'Longitude', 'xt_ocean', 'xt']", 'replacement_dictionary': 'None', 'time': "['Time']"}
		 Rename                         {'names': {'t2m': "'2t'", 'u10': "'10u'", 'v10': "'10v'", 'siconc': "'ci'"}}

In [19]:
UandV['1984-01-01']  # ERA5 is an analysis product, so is indexed by its analysis time. This request
                     # fetches all of the data from a particular date

<xarray.Dataset> Size: 15GB
Dimensions:    (longitude: 1440, latitude: 721, level: 37, time: 24)
Coordinates:
  * longitude  (longitude) float32 6kB -180.0 -179.8 -179.5 ... 179.5 179.8
  * latitude   (latitude) float32 3kB 90.0 89.75 89.5 ... -89.5 -89.75 -90.0
  * level      (level) int32 148B 1 2 3 5 7 10 20 ... 875 900 925 950 975 1000
  * time       (time) datetime64[ns] 192B 1984-01-01 ... 1984-01-01T23:00:00
Data variables:
    u          (time, level, latitude, longitude) float64 7GB dask.array<chunksize=(4, 5, 405, 900), meta=np.ndarray>
    v          (time, level, latitude, longitude) float64 7GB dask.array<chunksize=(4, 5, 405, 900), meta=np.ndarray>
Attributes:
    Conventions:  CF-1.6
    license:      Licence to use Copernicus Products: https://apps.ecmwf.int/...
    summary:      ERA5 is the fifth generation ECMWF atmospheric reanalysis o...

# What this buys you

EDIT has abstracted the relationship between the data loading and the data source, in this case the filesystem. By writing a specific connector to the filesystem, it is possible to thereafter treat all data sources in a compatible and generic fashion, and also modularise the code which deals with filesystem connections (such as handling different filesystem layouts e.g. YYYY, YYMM, YYYMMDD layouts, or others)

It also applies standard coordinate naming, so that data sources can be more easily combined. It is common, for example for one data set to pick 'lat' and 'lon' instead of 'latitude' and 'longitude', or 't2m' instead of '2t' and suchlike. 

At this point, it would be possible to write a simple loop such as:

for every date in a training period:
   load it call a training update function

This is a valid use case. The next notebook, "Data Pipelines with EDIT", will explore how to make further use of these data objects for data processing and options for presentation to the ML training frameworks. 

The rest of this notebook will explore how to connect a novel data source.

